<a href="https://colab.research.google.com/github/bnriiitb/whisper-fine-tuning/blob/main/model_evaluation_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Dec  5 18:36:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg
!apt-get install libsndfile1

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [1,056 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRel

In [4]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-_u4wq9bw
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-_u4wq9bw
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 7.6 MB 6.2 MB/s 
  Created wheel for transformers: filename=transformers-4.26.0.dev0-py3-none-any.whl size=5878865 sha256=e80fe06f02eb6a2c1765077b730e6b0effcdb151da8026e823c5934a0412a4a6
  Stored in directory: /tmp/pip-ephem-wheel-cache-mkahwntv/wheels/42/68/45/c63edff61c292f2dfd4df4ef6522dcbecc603e7af82813c1d7
Successfully built transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
    

In [5]:
!wget https://storage.googleapis.com/test_public_bucket/labelled/Telugu_Chai_Bisket_Stories_16-08-2021_14-17.zip
!unzip Telugu_Chai_Bisket_Stories_16-08-2021_14-17.zip

--2022-12-05 18:38:04--  https://storage.googleapis.com/test_public_bucket/labelled/Telugu_Chai_Bisket_Stories_16-08-2021_14-17.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 108.177.98.128, 74.125.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 90658931 (86M) [application/zip]
Saving to: ‘Telugu_Chai_Bisket_Stories_16-08-2021_14-17.zip’

Telugu_Chai_Bisket_ 100%[===================>]  86.46M   100MB/s    in 0.9s    

2022-12-05 18:38:05 (100 MB/s) - ‘Telugu_Chai_Bisket_Stories_16-08-2021_14-17.zip’ saved [90658931/90658931]

Archive:  Telugu_Chai_Bisket_Stories_16-08-2021_14-17.zip
   creating: Chai_Bisket_Stories_16-08-2021_14-17/
  inflating: Chai_Bisket_Stories_16-08-2021_14-17/3_1_481file-idc4p8tJhhC0o.wav  
  inflating: Chai_Bisket_Stories_16-08-2021_14-17/10_16_596file-idIErLEb3RPs4.wav  
  inflating: Chai_Bisket_Stories_16-08-2021_1

In [6]:
import datasets
from datasets import load_dataset, DatasetDict, Dataset, Audio

ds_1 = load_dataset("/content/Chai_Bisket_Stories_16-08-2021_14-17")

ds_1 = ds_1['train']
print(ds_1)

ds_2 = load_dataset("json",data_files="/content/Chai_Bisket_Stories_16-08-2021_14-17/data.json")
ds_2 = ds_2['train']
print(ds_2)

final_dataset = datasets.concatenate_datasets([ds_1, ds_2], axis=1)
print(final_dataset)

train_test_dataset = final_dataset.train_test_split(test_size=0.30)
print(train_test_dataset)


train_test_dataset = train_test_dataset.remove_columns(['audioFilename','collectionSource', 'snr', 'duration', 'gender',])
print(train_test_dataset)

Resolving data files:   0%|          | 0/587 [00:00<?, ?it/s]

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset audiofolder downloaded and prepared to /root/.cache/huggingface/datasets/audiofolder/Chai_Bisket_Stories_16-08-2021_14-17-f9140ff64dffe449/0.0.0/6cbdd16f8688354c63b4e2a36e1585d05de285023ee6443ffd71c4182055c0fc. Subsequent calls will reuse this data.


...

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-dc6752bd93399151/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['audioFilename', 'collectionSource', 'snr', 'duration', 'gender', 'text'],
    num_rows: 585
})
Dataset({
    features: ['audio', 'audioFilename', 'collectionSource', 'snr', 'duration', 'gender', 'text'],
    num_rows: 585
})
DatasetDict({
    train: Dataset({
        features: ['audio', 'audioFilename', 'collectionSource', 'snr', 'duration', 'gender', 'text'],
        num_rows: 409
    })
    test: Dataset({
        features: ['audio', 'audioFilename', 'collectionSource', 'snr', 'duration', 'gender', 'text'],
        num_rows: 176
    })
})
DatasetDict({
    train: Dataset({
        features: ['audio', 'text'],
        num_rows: 409
    })
    test: Dataset({
        features: ['audio', 'text'],
        num_rows: 176
    })
})


In [7]:
# from transformers import WhisperFeatureExtractor
# from transformers import WhisperTokenizer
# from transformers import WhisperProcessor
# from datasets import Audio

# feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
# tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Telugu", task="transcribe")
# processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Telugu", task="transcribe")

# common_voice = train_test_dataset.cast_column("audio", Audio(sampling_rate=16000))

In [8]:
# def prepare_dataset(batch):
#     # load and resample audio data from 48 to 16kHz
#     audio = batch["audio"]

#     # compute log-Mel input features from input audio array 
#     batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

#     # encode target text to label ids 
#     batch["labels"] = tokenizer(batch["text"]).input_ids
#     # batch["labels"] = tokenizer(batch["sentence"]).input_ids
#     return batch

# train_test_dataset = train_test_dataset.map(prepare_dataset, remove_columns=train_test_dataset.column_names["train"], num_proc=8)    

In [9]:
from transformers import pipeline
import gradio as gr

pipe = pipeline(model="bnriiitb/whisper-small-te")  # change to "your-username/the-name-you-picked"

# def transcribe(audio):
#     text = pipe(audio)["text"]
#     return text

# iface = gr.Interface(
#     fn=transcribe, 
#     inputs=gr.Audio(source="microphone", type="filepath"), 
#     outputs="text",
#     title="Whisper Small Telugu",
#     description="Realtime demo for Telugu speech recognition using a fine-tuned Whisper small model.",
# )

# iface.launch()

Downloading:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/967M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/829 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/494k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [10]:
# dataset = load_dataset("/content/Chai_Bisket_Stories_16-08-2021_14-17")
dataset = load_dataset("json",data_files="/content/Chai_Bisket_Stories_16-08-2021_14-17/data.json")
dataset

  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['audioFilename', 'collectionSource', 'snr', 'duration', 'gender', 'text'],
        num_rows: 585
    })
})

In [11]:
from IPython.display import Audio
file_name = '7_381file-id_j-j_NR0T1Y.wav'
Audio(f"/content/Chai_Bisket_Stories_16-08-2021_14-17/{file_name}")

In [12]:
dataset['train'][66]

{'audioFilename': '7_381file-id_j-j_NR0T1Y.wav',
 'collectionSource': ['Chai_Bisket_Stories',
  'https://www.youtube.com/watch?v=_j-j_NR0T1Y',
  'https://www.youtube.com/channel/UCmFcc_FIZkXO1ip-Qsg-KrA'],
 'snr': {'methodDetails': {'snr': 16.0}, 'methodType': 'WadaSnr'},
 'duration': 5.16,
 'gender': 'female',
 'text': 'ఇఫ్ యు టాక్ లైక్ దిస్   గర్ల్   విల్సన్'}

In [13]:
train_test_dataset['train'][20]

{'audio': {'path': '/content/Chai_Bisket_Stories_16-08-2021_14-17/2_747file-idBbnm8KVeyXE.wav',
  'array': array([-0.00317383,  0.00271606,  0.00201416, ..., -0.01257324,
         -0.00946045, -0.00613403], dtype=float32),
  'sampling_rate': 16000},
 'text': 'అప్పుడు అనుకోలేదు'}

In [14]:
Audio(train_test_dataset['train'][20]['audio']['path'])

In [15]:
pipe(f"/content/Chai_Bisket_Stories_16-08-2021_14-17/{file_name}",language='Telugu',device=2,task='transcribe')

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'text': ' Or hey, if you talk like this to any girl ever, I will put tacit on you'}

In [16]:
!pip install git+https://github.com/openai/whisper.git -q
!pip install -U tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
import whisper
whisper_small_model = whisper.load_model("small")

100%|████████████████████████████████████████| 461M/461M [00:02<00:00, 183MiB/s]


In [18]:
from IPython.display import Audio

info = dataset['train'][156]
print(info['text'])
file_name = info['audioFilename']
Audio(f"/content/Chai_Bisket_Stories_16-08-2021_14-17/{file_name}")

ప్రతి గుర్తొస్తుంది ఎగ్జాక్ట్ నైన్త్ క్లాస్  వచ్చేసరికి


In [19]:
# whisper_small_model.transcribe(f"/content/Chai_Bisket_Stories_16-08-2021_14-17/{file_name}",language="Telugu")

In [20]:
# curl -X GET "https://datasets-server.huggingface.co/first-rows?dataset=google%2Ffleurs&config=te_in&split=test"
fleurs_dataset = load_dataset("google/fleurs", "te_in", split="validation",num_proc=20)
fleurs_dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset fleurs downloaded and prepared to /root/.cache/huggingface/datasets/google___fleurs/te_in/2.0.0/aabb39fb29739c495517ac904e2886819b6e344702f0a5b5283cb178b087c94a. Subsequent calls will reuse this data.


Dataset({
    features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
    num_rows: 311
})

In [21]:
test_dataset = fleurs_dataset.remove_columns( ['id', 'num_samples', 'path',  'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'])
test_dataset

Dataset({
    features: ['audio', 'transcription'],
    num_rows: 311
})

In [22]:
reference_texts = test_dataset['transcription']
reference_texts[:2]

['వర్చువల్ ఫీల్డ్ ట్రిప్స్ తో టెక్నాలజీ పరిష్కారాన్ని అందిస్తుంది విద్యార్థులు మ్యూజియం కళాఖండాలను చూడవచ్చు అక్వేరియంను సందర్శించవచ్చు లేదా వాళ్ళ క్లాసులో కూర్చుని అందమైన కళను ఆరాధించవచ్చు',
 'ప్లాటింగ్ విశ్లేషణ ఫలితం పబ్లిక్ వెబ్సైట్లో పోస్ట్ చేయబడుతుంది']

In [2]:
from transformers import pipeline
# import gradio as gr

# pipe = pipeline(model="bnriiitb/whisper-small-te")  # change to "your-username/the-name-you-picked"

def transcribe(audio):
    text = pipe(audio)["text"]
    return text

import time
from tqdm import tqdm

In [3]:
pipe = pipeline(model="bnriiitb/whisper-small-te")  # change to "your-username/the-name-you-picked"
# pipe = pipeline(model="bnriiitb/whisper-small-te",language='Telugu')  # change to "your-username/the-name-you-picked"

Downloading: 100%|████████████████████████████████████████████████████████████████████████████| 829/829 [00:00<00:00, 492kB/s]
Downloading: 100%|████████████████████████████████████████████████████████████████████████| 1.04M/1.04M [00:01<00:00, 798kB/s]
Downloading: 100%|██████████████████████████████████████████████████████████████████████████| 494k/494k [00:01<00:00, 433kB/s]
Downloading: 100%|████████████████████████████████████████████████████████████████████████| 52.7k/52.7k [00:00<00:00, 116kB/s]
Downloading: 100%|███████████████████████████████████████████████████████████████████████| 2.11k/2.11k [00:00<00:00, 1.07MB/s]
Downloading: 100%|████████████████████████████████████████████████████████████████████████| 2.06k/2.06k [00:00<00:00, 952kB/s]
Downloading: 100%|██████████████████████████████████████████████████████████████████████████| 185k/185k [00:00<00:00, 294kB/s]


In [6]:
from IPython import display

In [8]:
!pwd

/raid/cs20mds14030/telugu_asr/notebooks


In [119]:
finetuned_texts = []

for item in tqdm(test_dataset['audio'][:5]):
  finetuned_texts.append(pipe(item['array'],language='Telugu')['text'])

finetuned_texts

  0%|          | 0/5 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 5/5 [00:50<00:00, 10.15s/it]


[' Virtual field trips are the main reason for the technological development. The museum is a place where you can visit and enjoy the views of the museum.',
 ' Posts are being posted on the public website',
 ' The local authorities have been working on the land for a long time. The air condition is not good.',
 ' I am Svante Desamla, from Tiruginath village, Kursh.',
 ' The']

In [135]:
# finetuned_texts.append(pipe(item['array'],language='Telugu',task='translate')['text'])
pipe(test_dataset['audio'][0]['array'],task='transcribe')

{'text': ' Virtual field trips are the main reason for the technological development. The museum is a place where you can visit and enjoy the views of the museum.'}

In [134]:
pipe(test_dataset['audio'][0]['array'],task='translate')

{'text': ' Virtual field trips are the main reason for the technological development. The museum is a place where you can visit and enjoy the views of the museum.'}

In [142]:
decode_options = {}
decode_options['language']='Telugu'
decode_options

{'language': 'Telugu'}

In [148]:
whisper_small_model.transcribe(fleurs_dataset['path'][0],task='translate',**decode_options)['text']

' With virtual field trips technology testing is performed Students can watch museum and see the museum They can study aquariums Or they can sit in class and study beautiful things'

In [120]:
fleurs_dataset['transcription'][:2]

['వర్చువల్ ఫీల్డ్ ట్రిప్స్ తో టెక్నాలజీ పరిష్కారాన్ని అందిస్తుంది విద్యార్థులు మ్యూజియం కళాఖండాలను చూడవచ్చు అక్వేరియంను సందర్శించవచ్చు లేదా వాళ్ళ క్లాసులో కూర్చుని అందమైన కళను ఆరాధించవచ్చు',
 'ప్లాటింగ్ విశ్లేషణ ఫలితం పబ్లిక్ వెబ్సైట్లో పోస్ట్ చేయబడుతుంది']

In [121]:
import whisper
whisper_small_model = whisper.load_model("small",device='cuda')

whisper_langs = []
whisper_texts = []

for item in tqdm(fleurs_dataset['path'][:5]):
  pred = whisper_small_model.transcribe(item)
  whisper_texts.append(pred['text'])
  whisper_langs.append(pred['language'])
  

whisper_texts

100%|██████████| 5/5 [01:15<00:00, 15.04s/it]


[' Trevor',
 ' ఆరిప్కాశ Св Worlds',
 ' నార్న్యార్నిక్నినినినారినినార్ని.',
 '',
 '']

In [123]:
import evaluate
metric = evaluate.load("wer")

finetuned_wer = 100 * metric.compute(predictions=finetuned_texts, references=fleurs_dataset['transcription'][:5])
whisper_wer = 100 * metric.compute(predictions=whisper_texts, references=fleurs_dataset['transcription'][:5])

whisper_wer,finetuned_wer

(100.0, 114.28571428571428)

In [ ]:
# fleurs_dataset['path'][0]
# fleurs_dataset['transcription'][0]

In [ ]:
# path = '/root/.cache/huggingface/datasets/downloads/extracted/eff9a77480c302763e814c669f231be745f4bd5002270e738bc2d0af8b19d934/te_in/audio/dev/9606134904533130568.wav'

In [ ]:
# Audio(path)

In [ ]:
# pipe(path,language='te')
# ['text']

In [ ]:
# whisper_small_model.transcribe(path,language='Telugu')['text']

In [ ]:
# whisper_small_model.transcribe(path)['text']

In [ ]:
# fleurs_dataset

In [ ]:
# # ds = load_dataset("common_voice", "fr", split="test", streaming=True)

# from datasets import load_dataset
# from transformers import WhisperForConditionalGeneration, WhisperProcessor
# import soundfile as sf
# import torch
# from evaluate import load

# model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small").to("cuda")
# processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Telugu", task="transcribe")

# # ds = fleurs_dataset.cast_column("path", Audio())
# input_speech = next(iter(fleurs_dataset))["audio"]["array"]

# # model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language = "te", task = "transcribe")
# input_features = processor(input_speech, return_tensors="pt").input_features 
# predicted_ids = model.generate(input_features)
# transcription = processor.batch_decode(predicted_ids)

In [ ]:
# # fleurs_dataset[0]['audio']['array']
# input_features = processor(fleurs_dataset[0]['audio']['array'], return_tensors="pt").input_features 
# predicted_ids = model.generate(input_features)
# transcription = processor.batch_decode(predicted_ids)

In [ ]:
# from datasets import load_dataset
# from transformers import WhisperForConditionalGeneration, WhisperProcessor
# import soundfile as sf
# import torch
# from evaluate import load


# # librispeech_eval = load_dataset("librispeech_asr", "clean", split="test")

# librispeech_eval = test_dataset

# model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small").to("cuda")
# processor = WhisperProcessor.from_pretrained("openai/whisper-small")

# def map_to_pred(batch):
#     input_features = processor(batch["audio"][0]["array"], return_tensors="pt").input_features

#     with torch.no_grad():
#         logits = model(input_features.to("cuda")).logits

#     predicted_ids = torch.argmax(logits, dim=-1)
#     transcription = processor.batch_decode(predicted_ids, normalize = True)
#     batch['text'] = processor.tokenizer._normalize(batch['text'])
#     batch["transcription"] = transcription
#     return batch

# result = librispeech_eval.map(map_to_pred, batched=True, batch_size=1)
# wer = load("wer")
# print(wer.compute(predictions=ds["text"], references=ds["transcription"]))